<a href="https://colab.research.google.com/github/jetshaggy85/info5731/blob/master/Fellers_selfstudy10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import division
from collections import Counter, defaultdict
import math, random, re, glob
from functools import partial

In [0]:
def split_data(data, prob):
  """split data into fractions [prob, 1 - prob]"""
  results = [], []
  for row in data:
    results[0 if random.random() < prob else 1].append(row)
  return results

In [0]:
def train_test_split(x, y, test_pct):
  data = zip(x, y) # pair corresponding values
  train, test = split_data(data, 1 - test_pct) # split the data set of pairs
  x_train, y_train = zip(*train) # magical un-zip trick
  x_test, y_test = zip(*test)
  return x_train, x_test, y_train, y_test

In [6]:
def accuracy(tp, fp, fn, tn):
  correct = tp + tn
  total = tp + fp + fn + tn
  return correct / total
print(accuracy(70, 4930, 13930, 981070))

0.98114


In [7]:
def precision(tp, fp, fn, tn):
  return tp / (tp + fp)
print(precision(70, 4930, 13930, 981070))

0.014


In [8]:
def recall(tp, fp, fn, tn):
  return tp / (tp + fn)
print(recall(70, 4930, 13930, 981070))

0.005


In [0]:
def f1_score(tp, fp, fn, tn):
  p = precision(tp, fp, fn, tn)
  r = recall(tp, fp, fn, tn)
  return 2 * p * r / (p + r)

In [0]:
def tokenize(message):
  message = message.lower() # convert to lowercase
  all_words = re.findall("[a-z0-9']+", message) # extract the words
  return set(all_words) # remove duplicates

In [0]:
def count_words(training_set):
  """training set consists of pairs (message, is_spam)"""
  counts = defaultdict(lambda: [0, 0])
  for message, is_spam in training_set:
    for word in tokenize(message):
      counts[word][0 if is_spam else 1] += 1
  return counts

In [0]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
  """turn the word_counts into a list of triplets
  w, p(w | spam) and p(w | ~spam)"""
  return [(w,
    (spam + k) / (total_spams + 2 * k),
    (non_spam + k) / (total_non_spams + 2 * k))
    for w, (spam, non_spam) in counts.iteritems()]

In [0]:
def spam_probability(word_probs, message):
  message_words = tokenize(message)
  log_prob_if_spam = log_prob_if_not_spam = 0.0
  
  # iterate through each word in our vocabulary
  for word, prob_if_spam, prob_if_not_spam in word_probs:
    
    # if *word* appears in the message,
    # add the log probability of seeing it
    if word in message_words:
      log_prob_if_spam += math.log(prob_if_spam)
      log_prob_if_not_spam += math.log(prob_if_not_spam)
    
    # if *word* doesn't appear in the message
    # add the log probability of _not_ seeing it
    # which is log(1 - probability of seeing it)
    else:
      log_prob_if_spam += math.log(1.0 - prob_if_spam)
      log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

  prob_if_spam = math.exp(log_prob_if_spam)
  prob_if_not_spam = math.exp(log_prob_if_not_spam)
  return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [0]:
class NaiveBayesClassifier:

  def __init__(self, k=0.5):
    self.k = k
    self.word_probs = []

  def train(self, training_set):

    # count spam and non-spam messages
    num_spams = len([is_spam
      for message, is_spam in training_set
      if is_spam])
    num_non_spams = len(training_set) - num_spams

    # run training data through our "pipeline"
    word_counts = count_words(training_set)
    self.word_probs = word_probabilities(word_counts,
      num_spams,
      num_non_spams,
      self.k)
  def classify(self, message):
    return spam_probability(self.word_probs, message)

In [0]:
import glob, re

# modify the path with wherever you've put the files
path = r"C:\spam\*\*"

data = []

# glob.glob returns every filename that matches the wildcarded path
for fn in glob.glob(path):
  is_spam = "ham" not in fn
  
  with open(fn,'r') as file:
    for line in file:
      if line.startswith("Subject:"):
        # remove the leading "Subject: " and keep what's left
        subject = re.sub(r"^Subject: ", "", line).strip()
        data.append((subject, is_spam))

In [0]:
  # triplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject))
  for subject, is_spam in test_data]

# assume that spam_probability > 0.5 corresponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)
counts = Counter((is_spam, spam_probability > 0.5)
  for _, is_spam, spam_probability in classified)

In [0]:
def p_spam_given_word(word_prob):
  """uses bayes's theorem to compute p(spam | message contains word)"""
  
  # word_prob is one of the triplets produced by word_probabilities
  word, prob_if_spam, prob_if_not_spam = word_prob
  return prob_if_spam / (prob_if_spam + prob_if_not_spam)
words = sorted(classifier.word_probs, key=p_spam_given_word)

spammiest_words = words[-5:]
hammiest_words = words[:5]

In [0]:
def drop_final_s(word):
  return re.sub("s$", "", word)

In [0]:
def entropy(class_probabilities):
  """given a list of class probabilities, compute the entropy"""
  return sum(-p * math.log(p, 2)
    for p in class_probabilities
    if p) # ignore zero probabilities

In [0]:
def class_probabilities(labels):
  total_count = len(labels)
  return [count / total_count
    for count in Counter(labels).values()]
def data_entropy(labeled_data):
  labels = [label for _, label in labeled_data]
  probabilities = class_probabilities(labels)
  return entropy(probabilities)

In [0]:
def partition_entropy(subsets):
  """find the entropy from this partition of data into subsets
  subsets is a list of lists of labeled data"""

  total_count = sum(len(subset) for subset in subsets)
  
  return sum( data_entropy(subset) * len(subset) / total_count
    for subset in subsets )

In [0]:
inputs = [
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False)
]

In [0]:
def partition_by(inputs, attribute):
  """each input is a pair (attribute_dict, label).
  returns a dict : attribute_value -> inputs"""
  groups = defaultdict(list)
  for input in inputs:
    key = input[0][attribute] # get the value of the specified attribute
    groups[key].append(input) # then add this input to the correct list
  return groups

In [0]:
def partition_entropy_by(inputs, attribute):
  """computes the entropy corresponding to the given partition"""
  partitions = partition_by(inputs, attribute)
  return partition_entropy(partitions.values())

In [46]:
for key in ['level','lang','tweets','phd']:
  print(key, partition_entropy_by(inputs, key))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [47]:
senior_inputs = [(input, label)
  for input, label in inputs if input["level"] == "Senior"]

for key in ['lang', 'tweets', 'phd']:
  print(key, partition_entropy_by(senior_inputs, key))

lang 0.4
tweets 0.0
phd 0.9509775004326938


In [0]:
def classify(tree, input):
  """classify the input using the given decision tree"""
  
  # if this is a leaf node, return its value
  if tree in [True, False]:
    return tree
  
  # otherwise this tree consists of an attribute to split on
  # and a dictionary whose keys are values of that attribute
  # and whose values of are subtrees to consider next
  attribute, subtree_dict = tree

  subtree_key = input.get(attribute) # None if input is missing attribute
  
  if subtree_key not in subtree_dict: # if no subtree for key,
    subtree_key = None # we'll use the None subtree
  
  subtree = subtree_dict[subtree_key] # choose the appropriate subtree
  return classify(subtree, input) # and use it to classify the input

In [0]:
def build_tree_id3(inputs, split_candidates=None):
  
  # if this is our first pass,
  # all keys of the first input are split candidates
  if split_candidates is None:
    split_candidates = inputs[0][0].keys()
  
  # count Trues and Falses in the inputs
  num_inputs = len(inputs)
  num_trues = len([label for item, label in inputs if label])
  num_falses = num_inputs - num_trues

  if num_trues == 0: return False # no Trues? return a "False" leaf
  if num_falses == 0: return True # no Falses? return a "True" leaf

  if not split_candidates: # if no split candidates left
    return num_trues >= num_falses # return the majority leaf

  # otherwise, split on the best attribute
  best_attribute = min(split_candidates,
    key=partial(partition_entropy_by, inputs))
  
  partitions = partition_by(inputs, best_attribute)
  new_candidates = [a for a in split_candidates
    if a != best_attribute]

  # recursively build the subtrees
  subtrees = { attribute_value : build_tree_id3(subset, new_candidates)
    for attribute_value, subset in partitions.items() }

  subtrees[None] = num_trues > num_falses # default case

  return (best_attribute, subtrees)

In [55]:
tree = build_tree_id3(inputs)

classify(tree, { "level" : "Junior",
  "lang" : "Java",
  "tweets" : "yes",
  "phd" : "no"} ) # True

classify(tree, { "level" : "Junior",
  "lang" : "Java",
  "tweets" : "yes",
  "phd" : "yes"} ) # False

False

In [56]:
classify(tree, { "level" : "Intern" } ) # True
classify(tree, { "level" : "Senior" } ) # False

False

In [0]:
def forest_classify(trees, input):
  votes = [classify(tree, input) for tree in trees]
  vote_counts = Counter(votes)
  return vote_counts.most_common(1)[0][0]

In [74]:
>>> def gender_features(word):
...     return {'last_letter': word[-1]}
>>> gender_features('Shrek')

{'last_letter': 'k'}

In [70]:
import nltk
>>> nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [93]:
 >>> import nltk
 >>> nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [101]:
 >>> import nltk
 >>> nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [0]:
>>> from nltk.corpus import names
>>> labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
... [(name, 'female') for name in names.words('female.txt')])
>>> import random
>>> random.shuffle(labeled_names)

In [0]:
>>> featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
>>> train_set, test_set = featuresets[500:], featuresets[:500]
>>> classifier = nltk.NaiveBayesClassifier.train(train_set)

In [75]:
>>> classifier.classify(gender_features('Neo'))
>>> classifier.classify(gender_features('Trinity'))

'female'

In [76]:
>>> print(nltk.classify.accuracy(classifier, test_set))

0.768


In [77]:
>>> classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     35.7 : 1.0
             last_letter = 'k'              male : female =     30.8 : 1.0
             last_letter = 'p'              male : female =     20.9 : 1.0
             last_letter = 'f'              male : female =     15.3 : 1.0
             last_letter = 'v'              male : female =     10.5 : 1.0


In [0]:
>>> from nltk.classify import apply_features
>>> train_set = apply_features(gender_features, labeled_names[500:])
>>> test_set = apply_features(gender_features, labeled_names[:500])

In [0]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [81]:
gender_features2('John')

{'count(a)': 0,
 'count(b)': 0,
 'count(c)': 0,
 'count(d)': 0,
 'count(e)': 0,
 'count(f)': 0,
 'count(g)': 0,
 'count(h)': 1,
 'count(i)': 0,
 'count(j)': 1,
 'count(k)': 0,
 'count(l)': 0,
 'count(m)': 0,
 'count(n)': 1,
 'count(o)': 1,
 'count(p)': 0,
 'count(q)': 0,
 'count(r)': 0,
 'count(s)': 0,
 'count(t)': 0,
 'count(u)': 0,
 'count(v)': 0,
 'count(w)': 0,
 'count(x)': 0,
 'count(y)': 0,
 'count(z)': 0,
 'first_letter': 'j',
 'has(a)': False,
 'has(b)': False,
 'has(c)': False,
 'has(d)': False,
 'has(e)': False,
 'has(f)': False,
 'has(g)': False,
 'has(h)': True,
 'has(i)': False,
 'has(j)': True,
 'has(k)': False,
 'has(l)': False,
 'has(m)': False,
 'has(n)': True,
 'has(o)': True,
 'has(p)': False,
 'has(q)': False,
 'has(r)': False,
 'has(s)': False,
 'has(t)': False,
 'has(u)': False,
 'has(v)': False,
 'has(w)': False,
 'has(x)': False,
 'has(y)': False,
 'has(z)': False,
 'last_letter': 'n'}

In [82]:
>>> featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
>>> train_set, test_set = featuresets[500:], featuresets[:500]
>>> classifier = nltk.NaiveBayesClassifier.train(train_set)
>>> print(nltk.classify.accuracy(classifier, test_set))

0.802


In [0]:
>>> train_names = labeled_names[1500:]
>>> devtest_names = labeled_names[500:1500]
>>> test_names = labeled_names[:500]

In [84]:
>>> train_set = [(gender_features(n), gender) for (n, gender) in train_names]
>>> devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
>>> test_set = [(gender_features(n), gender) for (n, gender) in test_names]
>>> classifier = nltk.NaiveBayesClassifier.train(train_set)
>>> print(nltk.classify.accuracy(classifier, devtest_set))

0.766


In [0]:
>>> errors = []
>>> for (name, tag) in devtest_names:
...     guess = classifier.classify(gender_features(name))
...     if guess != tag:
...         errors.append( (tag, guess, name) )

In [88]:
>>> for (tag, guess, name) in sorted(errors):
...     print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Aeriell                       
correct=female   guess=male     name=Agnes                         
correct=female   guess=male     name=Alis                          
correct=female   guess=male     name=Angel                         
correct=female   guess=male     name=Annabal                       
correct=female   guess=male     name=Ardys                         
correct=female   guess=male     name=Avis                          
correct=female   guess=male     name=Avril                         
correct=female   guess=male     name=Ayn                           
correct=female   guess=male     name=Bren                          
correct=female   guess=male     name=Brit                          
correct=female   guess=male     name=Brooks                        
correct=female   guess=male     name=Carleen                       
correct=female   guess=male     name=Carlynn                       
correct=female   guess=male     name=Carmon     

In [0]:
>>> def gender_features(word):
...     return {'suffix1': word[-1:],
...             'suffix2': word[-2:]}

In [90]:
>>> train_set = [(gender_features(n), gender) for (n, gender) in train_names]
>>> devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
>>> classifier = nltk.NaiveBayesClassifier.train(train_set)
>>> print(nltk.classify.accuracy(classifier, devtest_set))

0.787


In [0]:
>>> from nltk.corpus import movie_reviews
>>> documents = [(list(movie_reviews.words(fileid)), category)
...              for category in movie_reviews.categories()
...              for fileid in movie_reviews.fileids(category)]
>>> random.shuffle(documents)

In [0]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [96]:
>>> print(document_features(movie_reviews.words('pos/cv957_8737.txt'))) 

{'contains(plot)': True, 'contains(:)': True, 'contains(two)': True, 'contains(teen)': False, 'contains(couples)': False, 'contains(go)': False, 'contains(to)': True, 'contains(a)': True, 'contains(church)': False, 'contains(party)': False, 'contains(,)': True, 'contains(drink)': False, 'contains(and)': True, 'contains(then)': True, 'contains(drive)': False, 'contains(.)': True, 'contains(they)': True, 'contains(get)': True, 'contains(into)': True, 'contains(an)': True, 'contains(accident)': False, 'contains(one)': True, 'contains(of)': True, 'contains(the)': True, 'contains(guys)': False, 'contains(dies)': False, 'contains(but)': True, 'contains(his)': True, 'contains(girlfriend)': True, 'contains(continues)': False, 'contains(see)': False, 'contains(him)': True, 'contains(in)': True, 'contains(her)': False, 'contains(life)': False, 'contains(has)': True, 'contains(nightmares)': False, 'contains(what)': True, "contains(')": True, 'contains(s)': True, 'contains(deal)': False, 'contains

In [0]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [98]:
>>> print(nltk.classify.accuracy(classifier, test_set))

0.78


In [99]:
>>> classifier.show_most_informative_features(5)

Most Informative Features
     contains(stretched) = True              neg : pos    =      9.6 : 1.0
 contains(unimaginative) = True              neg : pos    =      7.7 : 1.0
    contains(schumacher) = True              neg : pos    =      7.4 : 1.0
        contains(sexist) = True              neg : pos    =      7.0 : 1.0
     contains(atrocious) = True              neg : pos    =      6.6 : 1.0


In [0]:
>>> from nltk.corpus import brown
>>> suffix_fdist = nltk.FreqDist()
>>> for word in brown.words():
...     word = word.lower()
...     suffix_fdist[word[-1:]] += 1
...     suffix_fdist[word[-2:]] += 1
...     suffix_fdist[word[-3:]] += 1

In [103]:
	
>>> common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
>>> print(common_suffixes)

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


In [0]:
>>> def pos_features(word):
...     features = {}
...     for suffix in common_suffixes:
...         features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
...     return features

In [0]:
>>> tagged_words = brown.tagged_words(categories='news')
>>> featuresets = [(pos_features(n), g) for (n,g) in tagged_words]

In [0]:
>>> size = int(len(featuresets) * 0.1)
>>> train_set, test_set = featuresets[size:], featuresets[:size]

In [0]:
#>>> classifier = nltk.DecisionTreeClassifier.train(train_set)
#>>> nltk.classify.accuracy(classifier, test_set)

In [111]:
>>> classifier.classify(pos_features('cats'))

'neg'

In [0]:
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

In [116]:
>>> pos_features(brown.sents()[0], 8)

{'prev-word': 'an', 'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion'}

In [0]:
>>> tagged_sents = brown.tagged_sents(categories='news')
>>> featuresets = []
>>> for tagged_sent in tagged_sents:
...     untagged_sent = nltk.tag.untag(tagged_sent)
...     for i, (word, tag) in enumerate(tagged_sent):
...         featuresets.append( (pos_features(untagged_sent, i), tag) )

>>> size = int(len(featuresets) * 0.1)
>>> train_set, test_set = featuresets[size:], featuresets[:size]
#>>> classifier = nltk.NaiveBayesClassifier.train(train_set)

#>>> nltk.classify.accuracy(classifier, test_set)

In [0]:
def pos_features(sentence, i, history):
     features = {"suffix(1)": sentence[i][-1:],
                 "suffix(2)": sentence[i][-2:],
                 "suffix(3)": sentence[i][-3:]}
     if i == 0:
         features["prev-word"] = "<START>"
         features["prev-tag"] = "<START>"
     else:
         features["prev-word"] = sentence[i-1]
         features["prev-tag"] = history[i-1]
     return features

class ConsecutivePosTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [121]:
>>> tagged_sents = brown.tagged_sents(categories='news')
>>> size = int(len(tagged_sents) * 0.1)
>>> train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
>>> tagger = ConsecutivePosTagger(train_sents)
>>> print(tagger.evaluate(test_sents))

0.7980528511821975


In [124]:
>>> import nltk
>>> nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [127]:
 >>> import nltk
 >>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
>>> sents = nltk.corpus.treebank_raw.sents()
>>> tokens = []
>>> boundaries = set()
>>> offset = 0
>>> for sent in sents:
...     tokens.extend(sent)
...     offset += len(sent)
...     boundaries.add(offset-1)

In [0]:
>>> def punct_features(tokens, i):
...     return {'next-word-capitalized': tokens[i+1][0].isupper(),
...             'prev-word': tokens[i-1].lower(),
...             'punct': tokens[i],
...             'prev-word-is-one-char': len(tokens[i-1]) == 1}

In [0]:
>>> featuresets = [(punct_features(tokens, i), (i in boundaries))
...                for i in range(1, len(tokens)-1)
...                if tokens[i] in '.?!']

In [131]:
>>> size = int(len(featuresets) * 0.1)
>>> train_set, test_set = featuresets[size:], featuresets[:size]
>>> classifier = nltk.NaiveBayesClassifier.train(train_set)
>>> nltk.classify.accuracy(classifier, test_set)

0.936026936026936

In [0]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

In [134]:
>>> import nltk
>>> nltk.download('nps_chat')

[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Unzipping corpora/nps_chat.zip.


True

In [0]:
>>> posts = nltk.corpus.nps_chat.xml_posts()[:10000]

In [0]:
>>> def dialogue_act_features(post):
...     features = {}
...     for word in nltk.word_tokenize(post):
...         features['contains({})'.format(word.lower())] = True
...     return features

In [138]:
>>> featuresets = [(dialogue_act_features(post.text), post.get('class'))
...                for post in posts]
>>> size = int(len(featuresets) * 0.1)
>>> train_set, test_set = featuresets[size:], featuresets[:size]
>>> classifier = nltk.NaiveBayesClassifier.train(train_set)
>>> print(nltk.classify.accuracy(classifier, test_set))

0.668


In [0]:
def rte_features(rtepair):
    extractor = nltk.RTEFeatureExtractor(rtepair)
    features = {}
    features['word_overlap'] = len(extractor.overlap('word'))
    features['word_hyp_extra'] = len(extractor.hyp_extra('word'))
    features['ne_overlap'] = len(extractor.overlap('ne'))
    features['ne_hyp_extra'] = len(extractor.hyp_extra('ne'))
    return features

In [141]:
 >>> import nltk
 >>> nltk.download('rte')

[nltk_data] Downloading package rte to /root/nltk_data...
[nltk_data]   Unzipping corpora/rte.zip.


True

In [143]:
>>> rtepair = nltk.corpus.rte.pairs(['rte3_dev.xml'])[33]
>>> extractor = nltk.RTEFeatureExtractor(rtepair)
>>> print(extractor.text_words)
>>> print(extractor.hyp_words)
>>> print(extractor.overlap('word'))
>>> print(extractor.overlap('ne'))
>>> print(extractor.hyp_extra('word'))

{'together', 'Asia', 'Co', 'China', 'was', 'meeting', 'fledgling', 'Russia', 'four', 'Organisation', 'Soviet', 'Iran', 'Davudi', 'central', 'former', 'operation', 'Shanghai', 'association', 'representing', 'terrorism.', 'republics', 'that', 'fight', 'SCO', 'Parviz', 'at', 'binds'}
{'member', 'China', 'SCO.'}
set()
{'China'}
{'member'}


In [0]:
>>> import random
>>> from nltk.corpus import brown
>>> tagged_sents = list(brown.tagged_sents(categories='news'))
>>> random.shuffle(tagged_sents)
>>> size = int(len(tagged_sents) * 0.1)
>>> train_set, test_set = tagged_sents[size:], tagged_sents[:size]

In [0]:
>>> file_ids = brown.fileids(categories='news')
>>> size = int(len(file_ids) * 0.1)
>>> train_set = brown.tagged_sents(file_ids[size:])
>>> test_set = brown.tagged_sents(file_ids[:size])

In [0]:
>>> train_set = brown.tagged_sents(categories='news')
>>> test_set = brown.tagged_sents(categories='fiction')

In [0]:
import math
def entropy(labels):
    freqdist = nltk.FreqDist(labels)
    probs = [freqdist.freq(l) for l in freqdist]
    return -sum(p * math.log(p,2) for p in probs)

In [151]:
>>> print(entropy(['male', 'male', 'male', 'male'])) 
>>> print(entropy(['male', 'female', 'male', 'male']))
>>> print(entropy(['female', 'male', 'female', 'male']))
>>> print(entropy(['female', 'female', 'male', 'female']))
>>> print(entropy(['female', 'female', 'female', 'female'])) 

-0.0
0.8112781244591328
1.0
0.8112781244591328
-0.0
